# 🏰 H-MOLQD: Hybrid Masked-diffusion Optimization with Logical Quality Diversity

**GPU-Accelerated Training Notebook for Zelda Dungeon Generation**

## Architecture Overview (6 Blocks)

```
VGLC Data (18 dungeons × .txt grids + .dot mission graphs)
    │
    ▼
┌──────────────────────────────────────────────────────────────────────┐
│  Block I  — VQ-VAE: 44 tiles → 512 codebook → latent_dim=64       │
│  Block II — Dual-Stream Condition Encoder (GATv2Conv GNN + Local)  │
│  Block III— Latent Diffusion (U-Net, cosine schedule, DDIM)        │
│  Block IV — LogicNet (differentiable solvability + key-lock check) │
│  Block V  — WFC Refiner (inference-only post-processing)           │
│  Block VI — Cognitive Validator (A* solver + CBS agent)            │
└──────────────────────────────────────────────────────────────────────┘
```

## Training Pipeline
- **Stage 1**: VQ-VAE pretraining (reconstructing dungeon grids)
- **Stage 2**: Latent Diffusion with **real .dot graph conditioning** + LogicNet guidance

## How to Use on Kaggle
1. Upload your project repo as a **Kaggle Dataset**
2. Enable **GPU accelerator** (T4/P100)
3. Run all cells sequentially
4. Download checkpoints from outputs

In [ ]:
# ============================================================================
# CELL 1: Environment Setup
# ============================================================================
import os, sys, subprocess
from pathlib import Path

IS_KAGGLE = os.path.exists('/kaggle/working')
IS_COLAB = 'google.colab' in sys.modules
ENV_NAME = "Kaggle" if IS_KAGGLE else ("Colab" if IS_COLAB else "Local")
print(f"🖥️  Environment: {ENV_NAME}")

# --- Find project root ---
if IS_KAGGLE:
    WORKING_DIR = Path('/kaggle/working')
    candidates = [
        Path('/kaggle/input/kltn'), Path('/kaggle/input/hmolqd'),
        Path('/kaggle/input/kltn/KLTN'), Path('/kaggle/working/KLTN'),
    ]
elif IS_COLAB:
    WORKING_DIR = Path('/content')
    candidates = [Path('/content/KLTN'), Path('/content/drive/MyDrive/KLTN')]
else:
    WORKING_DIR = Path('.').resolve()
    candidates = [Path('.').resolve(), Path('..').resolve()]

PROJECT_ROOT = None
for c in candidates:
    if (c / 'src' / 'train_vqvae.py').exists():
        PROJECT_ROOT = c
        break

if PROJECT_ROOT is None and not (not IS_KAGGLE and not IS_COLAB):
    clone_target = WORKING_DIR / 'KLTN'
    if not clone_target.exists():
        subprocess.run(['git', 'clone', 'https://github.com/YOUR_USERNAME/KLTN.git',
                        str(clone_target)], check=False)
    if (clone_target / 'src' / 'train_vqvae.py').exists():
        PROJECT_ROOT = clone_target

if PROJECT_ROOT is None:
    raise FileNotFoundError("❌ Could not find project root! Upload repo as Kaggle dataset.")

if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))
os.chdir(PROJECT_ROOT)

CHECKPOINT_DIR = (WORKING_DIR if IS_KAGGLE else PROJECT_ROOT) / 'checkpoints'
OUTPUT_DIR = (WORKING_DIR if IS_KAGGLE else PROJECT_ROOT) / 'output'
CHECKPOINT_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

DATA_DIR = PROJECT_ROOT / 'data' / 'The Legend of Zelda'
if not DATA_DIR.exists():
    DATA_DIR = PROJECT_ROOT / 'Data' / 'The Legend of Zelda'

print(f"✅ Project root: {PROJECT_ROOT}")
print(f"📁 Data dir: {DATA_DIR} (exists={DATA_DIR.exists()})")
print(f"💾 Checkpoints: {CHECKPOINT_DIR}")

# Install dependencies
req = PROJECT_ROOT / 'requirements-hmolqd.txt'
if req.exists():
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', '-r', str(req)], check=False)
print("🎉 Setup complete!")

In [ ]:
# ============================================================================
# CELL 2: Import Verification & GPU Check
# ============================================================================
import torch
import numpy as np
import matplotlib.pyplot as plt
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s | %(levelname)s | %(message)s', datefmt='%H:%M:%S')

print("=" * 60)
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)} ({torch.cuda.get_device_properties(0).total_mem / 1e9:.1f} GB)")
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {DEVICE}")

# Import project modules
from src.core.vqvae import SemanticVQVAE, create_vqvae, VQVAETrainer
from src.core.latent_diffusion import create_latent_diffusion
from src.core.condition_encoder import create_condition_encoder
from src.core.logic_net import LogicNet
from src.data.zelda_loader import create_dataloader, graph_collate_fn
from src.train_vqvae import grids_to_onehot
from src.train_diffusion import DiffusionTrainingConfig, DiffusionTrainer
print("✅ All modules imported")

# Quick data test
loader_test = create_dataloader(str(DATA_DIR), batch_size=2, use_vglc=True, normalize=True, load_graphs=True)
print(f"\n📊 Dataset: {len(loader_test.dataset)} dungeons")
for batch in loader_test:
    if isinstance(batch, (list, tuple)):
        imgs, graphs = batch
        g = graphs[0]
        print(f"   Image batch: {imgs.shape}")
        print(f"   Graph[0]: {g['num_nodes']} nodes, {g['num_edges']} edges, features={g['node_features'].shape}")
    break
print("✅ Data loading verified with real .dot graphs!")

---
## 🔵 Stage 1: VQ-VAE Pretraining (Block I)

Train the Semantic VQ-VAE to reconstruct dungeon grids.
- **Input**: 44-class one-hot tiles `[B, 44, H, W]`
- **Codebook**: 512 embeddings, latent_dim=64
- **Goal**: ≥85% reconstruction accuracy before Stage 2

In [ ]:
# ============================================================================
# CELL 3: Stage 1 — VQ-VAE Pretraining
# ============================================================================
import json, time
from torch.utils.data import DataLoader, RandomSampler

VQVAE_EPOCHS = 50          # Increase to 200-300 for production
VQVAE_BATCH_SIZE = 2
VQVAE_LR = 3e-4
MIN_SAMPLES_PER_EPOCH = 64
VQVAE_SAVE_PATH = CHECKPOINT_DIR / 'vqvae_pretrained.pth'

print("=" * 60)
print("🔵 STAGE 1: VQ-VAE PRETRAINING")
print("=" * 60)

# Check for existing checkpoint
SKIP_VQVAE = False
if VQVAE_SAVE_PATH.exists():
    ckpt = torch.load(VQVAE_SAVE_PATH, map_location='cpu', weights_only=False)
    existing_acc = ckpt.get('accuracy', 0)
    print(f"⚡ Found checkpoint: epoch={ckpt.get('epoch', 0)+1}, accuracy={existing_acc:.3f}")
    if existing_acc >= 0.85:
        print("   ✅ Accuracy ≥ 85% — skipping (delete checkpoint to retrain)")
        SKIP_VQVAE = True

if not SKIP_VQVAE:
    # Dataset with upsampling
    base_loader = create_dataloader(str(DATA_DIR), batch_size=VQVAE_BATCH_SIZE,
                                     shuffle=True, use_vglc=True, normalize=True, load_graphs=False)
    dataset = base_loader.dataset
    sampler = RandomSampler(dataset, replacement=True, num_samples=max(len(dataset), MIN_SAMPLES_PER_EPOCH))
    dataloader = DataLoader(dataset, batch_size=VQVAE_BATCH_SIZE, sampler=sampler, num_workers=0, drop_last=True)
    print(f"📊 {len(dataset)} dungeons, {len(dataloader)} batches/epoch (upsampled)")

    # Model
    vqvae_model = create_vqvae(num_classes=44, codebook_size=512, latent_dim=64).to(DEVICE)
    trainer = VQVAETrainer(vqvae_model, lr=VQVAE_LR)
    print(f"🏗️  VQ-VAE: {sum(p.numel() for p in vqvae_model.parameters()):,} params")

    best_loss, history = float('inf'), []
    t0 = time.time()

    for epoch in range(VQVAE_EPOCHS):
        vqvae_model.train()
        metrics_sum = {'loss': 0, 'recon_loss': 0, 'vq_loss': 0, 'perplexity': 0}
        nb = 0
        for batch in dataloader:
            if isinstance(batch, (list, tuple)): batch = batch[0]
            batch = batch.to(DEVICE)
            x_onehot = grids_to_onehot(batch, num_classes=44)
            _, m = trainer.train_step(x_onehot)
            for k in metrics_sum: metrics_sum[k] += m.get(k, 0.0)
            nb += 1
        for k in metrics_sum: metrics_sum[k] /= max(nb, 1)

        # Eval accuracy
        vqvae_model.eval()
        acc_sum, acc_n = 0, 0
        with torch.no_grad():
            for batch in dataloader:
                if isinstance(batch, (list, tuple)): batch = batch[0]
                x = grids_to_onehot(batch.to(DEVICE), num_classes=44)
                info = trainer.eval_step(x)
                acc_sum += info['accuracy']; acc_n += 1
                if acc_n >= 5: break
        eval_acc = acc_sum / max(acc_n, 1)
        metrics_sum['accuracy'] = eval_acc
        history.append({'epoch': epoch+1, **metrics_sum})

        if metrics_sum['loss'] < best_loss:
            best_loss = metrics_sum['loss']
            torch.save({'epoch': epoch, 'model_state_dict': vqvae_model.state_dict(),
                        'optimizer_state_dict': trainer.optimizer.state_dict(),
                        'loss': best_loss, 'accuracy': eval_acc,
                        'perplexity': metrics_sum['perplexity']}, VQVAE_SAVE_PATH)

        if (epoch+1) % 5 == 0 or epoch == 0:
            print(f"  Epoch {epoch+1:3d}/{VQVAE_EPOCHS} | loss={metrics_sum['loss']:.4f} | "
                  f"recon={metrics_sum['recon_loss']:.4f} | accuracy={eval_acc:.3f} | "
                  f"{time.time()-t0:.0f}s")

    print(f"\n✅ VQ-VAE done! Best loss={best_loss:.4f}, Final acc={eval_acc:.3f}")
    print(f"   Saved to: {VQVAE_SAVE_PATH}")

    # Plot
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    ep_x = [h['epoch'] for h in history]
    axes[0].plot(ep_x, [h['loss'] for h in history], 'b-'); axes[0].set_title('Loss'); axes[0].grid(True, alpha=0.3)
    axes[1].plot(ep_x, [h['recon_loss'] for h in history], 'r-', label='Recon')
    axes[1].plot(ep_x, [h['vq_loss'] for h in history], 'g-', label='VQ')
    axes[1].set_title('Components'); axes[1].legend(); axes[1].grid(True, alpha=0.3)
    axes[2].plot(ep_x, [h['accuracy'] for h in history], 'm-'); axes[2].set_title('Accuracy')
    axes[2].set_ylim(0, 1); axes[2].grid(True, alpha=0.3)
    plt.suptitle('Stage 1: VQ-VAE Training', fontweight='bold')
    plt.tight_layout(); plt.savefig(str(OUTPUT_DIR / 'vqvae_curves.png'), dpi=150); plt.show()
else:
    print("⏭️  Skipping VQ-VAE training")

---
## 🟢 Stage 2: Latent Diffusion with Real Graph Conditioning

Full pipeline training using **real .dot mission graphs** from VGLC:
- **U-Net** denoiser in VQ-VAE latent space
- **GATv2Conv GNN** encodes real dungeon graph topology (nodes=rooms, edges=doors)
- **LogicNet** gradient guidance: differentiable solvability + key-lock checking
- **EMA** model weights for stable sampling

| Phase | Epochs | Loss Components |
|-------|--------|-----------------|
| Warmup | 1–5 | Diffusion only (no logic) |
| Full | 6+ | Diffusion + α×LogicNet |

In [ ]:
# ============================================================================
# CELL 4: Stage 2 — Diffusion Training with Real Graph Conditioning
# ============================================================================
import time

DIFFUSION_EPOCHS = 500       # 100+ recommended; 500 for production
DIFFUSION_BATCH_SIZE = 2
DIFFUSION_LR = 1e-4
ALPHA_LOGIC = 0.1
WARMUP_EPOCHS = 5
SAVE_EVERY = 10

print("=" * 60)
print("🟢 STAGE 2: LATENT DIFFUSION TRAINING")
print("=" * 60)

if not VQVAE_SAVE_PATH.exists():
    raise FileNotFoundError(f"❌ VQ-VAE checkpoint not found. Run Stage 1 first!")

config = DiffusionTrainingConfig(
    data_dir=str(DATA_DIR), batch_size=DIFFUSION_BATCH_SIZE, use_vglc=True,
    vqvae_checkpoint=str(VQVAE_SAVE_PATH), latent_dim=64, model_channels=128,
    context_dim=256, num_timesteps=1000, schedule_type='cosine',
    num_logic_iterations=30, guidance_scale=1.0, epochs=DIFFUSION_EPOCHS,
    learning_rate=DIFFUSION_LR, alpha_visual=1.0, alpha_logic=ALPHA_LOGIC,
    warmup_epochs=WARMUP_EPOCHS, checkpoint_dir=str(CHECKPOINT_DIR),
    save_every=SAVE_EVERY, device='cuda' if torch.cuda.is_available() else 'cpu',
)

# Data loaders with REAL graph data from .dot files
train_loader = create_dataloader(config.data_dir, batch_size=config.batch_size,
    shuffle=True, use_vglc=True, normalize=True, load_graphs=True)
val_loader = create_dataloader(config.data_dir, batch_size=config.batch_size,
    shuffle=False, use_vglc=True, normalize=True, load_graphs=True)
print(f"📊 {len(train_loader.dataset)} dungeons with real .dot graphs, {len(train_loader)} batches/epoch")

# Create trainer
diff_trainer = DiffusionTrainer(config)
print(f"🏗️  All models on {config.device}, VQ-VAE frozen ✅")

# Resume from checkpoint if exists
resume_path = CHECKPOINT_DIR / 'final_model.pth'
start_epoch = 0
if resume_path.exists():
    try:
        diff_trainer.load_checkpoint(str(resume_path))
        start_epoch = diff_trainer.epoch
        print(f"⚡ Resumed from epoch {start_epoch}")
    except Exception as e:
        print(f"⚠️  Resume failed ({e}), starting fresh")

# Training loop
best_solv, history = 0.0, []
t0 = time.time()
print(f"\n{'Epoch':>6} | {'Loss':>8} | {'Diffusion':>10} | {'Logic':>8} | {'Val Solv':>10} | {'Time':>6}")
print("─" * 65)

for epoch in range(start_epoch, DIFFUSION_EPOCHS):
    train_m = diff_trainer.train_epoch(train_loader)
    val_m = diff_trainer.validate(val_loader, num_samples=4)
    lr = diff_trainer.scheduler.get_last_lr()[0]

    rec = {'epoch': epoch+1, **train_m, 'val_solvability': val_m['val_solvability'], 'lr': lr}
    history.append(rec)

    logic_flag = "🔒" if epoch < WARMUP_EPOCHS else "✅"
    elapsed = time.time() - t0
    print(f"  {epoch+1:4d}   | {train_m['loss']:.4f}  | {train_m['diffusion_loss']:.6f}  | "
          f"{train_m['logic_loss']:.4f}{logic_flag} | {val_m['val_solvability']:.4f}     | {elapsed/60:.1f}m")

    if (epoch+1) % SAVE_EVERY == 0:
        diff_trainer.save_checkpoint(str(CHECKPOINT_DIR / f'checkpoint_{epoch+1:04d}.pth'), rec)
    if val_m['val_solvability'] > best_solv:
        best_solv = val_m['val_solvability']
        diff_trainer.save_checkpoint(str(CHECKPOINT_DIR / 'best_model.pth'), rec)

diff_trainer.save_checkpoint(str(CHECKPOINT_DIR / 'final_model.pth'), history[-1] if history else None)
print(f"\n✅ Done! Best val solvability: {best_solv:.4f}, Time: {(time.time()-t0)/60:.1f} min")

with open(CHECKPOINT_DIR / 'diffusion_history.json', 'w') as f:
    json.dump(history, f, indent=2)

# Plot
if len(history) > 1:
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    ep_x = [h['epoch'] for h in history]
    axes[0,0].plot(ep_x, [h['loss'] for h in history], 'b-'); axes[0,0].set_title('Total Loss'); axes[0,0].grid(True, alpha=0.3)
    axes[0,1].plot(ep_x, [h['diffusion_loss'] for h in history], 'r-', label='Diffusion')
    axes[0,1].plot(ep_x, [h['logic_loss'] for h in history], 'g-', label='Logic')
    axes[0,1].axvline(x=WARMUP_EPOCHS, color='gray', ls='--', alpha=0.5, label='Warmup end')
    axes[0,1].set_title('Loss Components'); axes[0,1].legend(); axes[0,1].grid(True, alpha=0.3)
    axes[1,0].plot(ep_x, [h['solvability'] for h in history], 'c-', label='Train')
    axes[1,0].plot(ep_x, [h['val_solvability'] for h in history], 'm-', label='Val')
    axes[1,0].set_title('Solvability'); axes[1,0].legend(); axes[1,0].set_ylim(0,1); axes[1,0].grid(True, alpha=0.3)
    axes[1,1].plot(ep_x, [h['lr'] for h in history], 'k-'); axes[1,1].set_title('Learning Rate')
    axes[1,1].set_yscale('log'); axes[1,1].grid(True, alpha=0.3)
    plt.suptitle('Stage 2: Diffusion Training', fontweight='bold')
    plt.tight_layout(); plt.savefig(str(OUTPUT_DIR / 'diffusion_curves.png'), dpi=150); plt.show()

---
## 🎨 Generation & Visualization

Generate sample dungeons using the trained model with graph-conditioned DDIM sampling.

In [ ]:
# ============================================================================
# CELL 5: Generate Sample Dungeons
# ============================================================================
import matplotlib.colors as mcolors

NUM_SAMPLES = 4

print("🎨 GENERATING SAMPLE DUNGEONS")

# Load best checkpoint
ckpt_path = CHECKPOINT_DIR / 'best_model.pth'
if not ckpt_path.exists():
    ckpt_path = CHECKPOINT_DIR / 'final_model.pth'

gen_config = DiffusionTrainingConfig(
    data_dir=str(DATA_DIR), batch_size=1, use_vglc=True,
    vqvae_checkpoint=str(VQVAE_SAVE_PATH), latent_dim=64,
    model_channels=128, context_dim=256, num_timesteps=1000,
    schedule_type='cosine', device='cuda' if torch.cuda.is_available() else 'cpu',
)
gen = DiffusionTrainer(gen_config)
gen.load_checkpoint(str(ckpt_path))
print(f"✅ Loaded model (epoch {gen.epoch})")

# Get real graph conditioning
cond_loader = create_dataloader(str(DATA_DIR), batch_size=1, shuffle=True,
    use_vglc=True, normalize=True, load_graphs=True)
conditionings, shapes = [], []
for batch_data in cond_loader:
    if isinstance(batch_data, (list, tuple)) and len(batch_data) == 2:
        imgs, graphs = batch_data
        imgs = imgs.to(DEVICE)
        for g in graphs:
            try: conditionings.append(gen._encode_graph_conditioning(g))
            except: pass
        shapes.append(gen.encode_to_latent(imgs).shape)
    if len(conditionings) >= NUM_SAMPLES: break

# Generate
gen.ema_diffusion.eval()
generated = []
with torch.no_grad():
    for i in range(NUM_SAMPLES):
        c = conditionings[i % len(conditionings)]
        z = gen.ema_diffusion.sample(c, shape=shapes[0])
        logits = gen.decode_from_latent(z)
        tile_ids = logits.argmax(dim=1).squeeze(0).cpu().numpy()
        generated.append(tile_ids)
        print(f"  Sample {i+1}: {tile_ids.shape}, {len(np.unique(tile_ids))} tile types")

# Visualize
TILE_COLORS = {
    0: '#1a1a2e', 1: '#e8d5b7', 2: '#4a4a4a', 3: '#8b7355',
    10: '#90EE90', 11: '#FFD700', 12: '#FF6347', 13: '#9370DB',
    14: '#DC143C', 15: '#FFA500', 20: '#FF4444', 21: '#00FF00',
    22: '#FFD700', 23: '#8B0000', 30: '#FFFF00', 31: '#FF69B4',
    32: '#00CED1', 33: '#87CEEB', 40: '#4169E1', 41: '#6495ED',
    42: '#DEB887', 43: '#DA70D6',
}

def grid_to_rgb(grid):
    h, w = grid.shape
    rgb = np.full((h, w, 3), 0.5, dtype=np.float32)
    for tid, hex_c in TILE_COLORS.items():
        mask = grid == tid
        if mask.any():
            r, g, b = mcolors.hex2color(hex_c)
            rgb[mask] = [r, g, b]
    return rgb

fig, axes = plt.subplots(1, NUM_SAMPLES, figsize=(5*NUM_SAMPLES, 8))
if NUM_SAMPLES == 1: axes = [axes]
for i, (ax, grid) in enumerate(zip(axes, generated)):
    ax.imshow(grid_to_rgb(grid), interpolation='nearest', aspect='auto')
    ax.set_title(f'Dungeon {i+1}\n{grid.shape[0]}x{grid.shape[1]}')
    ax.axis('off')
plt.suptitle('H-MOLQD Generated Dungeons (Graph-Conditioned)', fontweight='bold')
plt.tight_layout()
plt.savefig(str(OUTPUT_DIR / 'generated_samples.png'), dpi=150)
plt.show()

for i, grid in enumerate(generated):
    np.save(str(OUTPUT_DIR / f'dungeon_{i+1}.npy'), grid)
print(f"💾 Saved {NUM_SAMPLES} dungeons to {OUTPUT_DIR}")

---

## 📦 Summary & Download

### Checkpoints Saved

| File | Description |
|------|-------------|
| `checkpoints/vqvae_pretrained.pth` | VQ-VAE encoder/decoder (44 tiles, 512 codebook) |
| `checkpoints/best_model.pth` | Best diffusion model (lowest val loss) |
| `checkpoints/final_model.pth` | Final diffusion model (last epoch) |

### Generated Outputs

| File | Description |
|------|-------------|
| `outputs/generated_samples.png` | Visualization of generated dungeons |
| `outputs/dungeon_*.npy` | Raw tile grids (NumPy arrays) |

### Download Results (Kaggle)

```python
import shutil
shutil.make_archive('/kaggle/working/hmolqd_results', 'zip', '/kaggle/working/hmolqd_outputs')
```

Then download `hmolqd_results.zip` from the **Output** tab.

### Next Steps (Local)
1. **WFC Refinement** (Block V): `python -m src.generation.wfc_refiner --input outputs/dungeon_1.npy`
2. **Cognitive Validation** (Block VI): `python -m src.simulation.cognitive_validator --dungeon outputs/dungeon_1.npy`
3. **Quality-Diversity Search**: Use MAP-Elites with trained models for diverse dungeon generation